In [37]:
print('check this: https://github.com/tableau/rest-api-samples')
import requests # Contains methods used to make HTTP requests
import xml.etree.ElementTree as ET # Contains methods used to build and parse XML
import sys
import os
import math
import getpass

# The following packages are used to build a multi-part/mixed request.
# They are contained in the 'requests' library
from requests.packages.urllib3.fields import RequestField
from requests.packages.urllib3.filepost import encode_multipart_formdata

# The namespace for the REST API is 'http://tableausoftware.com/api' for Tableau Server 9.0
# or 'http://tableau.com/api' for Tableau Server 9.1 or later
xmlns = {'t': 'http://tableau.com/api'}

# The maximum size of a file that can be published in a single request is 64MB
FILESIZE_LIMIT = 1024 * 1024 * 64   # 64MB

# For when a workbook is over 64MB, break it into 5MB(standard chunk size) chunks
CHUNK_SIZE = 1024 * 1024 * 5    # 5MB

# If using python version 3.x, 'raw_input()' is changed to 'input()'
if sys.version[0] == '3': raw_input=input



class ApiCallError(Exception):
    pass


class UserDefinedFieldError(Exception):
    pass


def _encode_for_display(text):
    """
    Encodes strings so they can display as ASCII in a Windows terminal window.
    This function also encodes strings for processing by xml.etree.ElementTree functions. 
    
    Returns an ASCII-encoded version of the text.
    Unicode characters are converted to ASCII placeholders (for example, "?").
    """
    return text.encode('ascii', errors="backslashreplace").decode('utf-8')


def _make_multipart(parts):
    """
    Creates one "chunk" for a multi-part upload
    'parts' is a dictionary that provides key-value pairs of the format name: (filename, body, content_type).
    Returns the post body and the content type string.
    For more information, see this post:
        http://stackoverflow.com/questions/26299889/how-to-post-multipart-list-of-json-xml-files-using-python-requests
    """
    mime_multipart_parts = []
    for name, (filename, blob, content_type) in parts.items():
        multipart_part = RequestField(name=name, data=blob, filename=filename)
        multipart_part.make_multipart(content_type=content_type)
        mime_multipart_parts.append(multipart_part)

    post_body, content_type = encode_multipart_formdata(mime_multipart_parts)
    content_type = ''.join(('multipart/mixed',) + content_type.partition(';')[1:])
    return post_body, content_type


def _check_status(server_response, success_code):
    """
    Checks the server response for possible errors.
    'server_response'       the response received from the server
    'success_code'          the expected success code for the response
    Throws an ApiCallError exception if the API call fails.
    """
    if server_response.status_code != success_code:
        parsed_response = ET.fromstring(server_response.text)

        # Obtain the 3 xml tags from the response: error, summary, and detail tags
        error_element = parsed_response.find('t:error', namespaces=xmlns)
        summary_element = parsed_response.find('.//t:summary', namespaces=xmlns)
        detail_element = parsed_response.find('.//t:detail', namespaces=xmlns)

        # Retrieve the error code, summary, and detail if the response contains them
        code = error_element.get('code', 'unknown') if error_element is not None else 'unknown code'
        summary = summary_element.text if summary_element is not None else 'unknown summary'
        detail = detail_element.text if detail_element is not None else 'unknown detail'
        error_message = '{0}: {1} - {2}'.format(code, summary, detail)
        raise ApiCallError(error_message)
    return

def sign_out(server, auth_token):
    """
    Destroys the active session and invalidates authentication token.
    'server'        specified server address
    'auth_token'    authentication token that grants user access to API calls
    """
    url = server + "/api/{0}/auth/signout".format(VERSION)
    server_response = requests.post(url, headers={'x-tableau-auth': auth_token})
    _check_status(server_response, 204)
    return


def start_upload_session(server, auth_token, site_id):
    """
    Creates a POST request that initiates a file upload session.
    'server'        specified server address
    'auth_token'    authentication token that grants user access to API calls
    'site_id'       ID of the site that the user is signed into
    Returns a session ID that is used by subsequent functions to identify the upload session.
    """
    url = server + "/api/{0}/sites/{1}/fileUploads".format(VERSION, site_id)
    server_response = requests.post(url, headers={'x-tableau-auth': auth_token})
    _check_status(server_response, 201)
    xml_response = ET.fromstring(_encode_for_display(server_response.text))
    return xml_response.find('t:fileUpload', namespaces=xmlns).get('uploadSessionId')


def get_default_project_id(server, auth_token, site_id):
    """
    Returns the project ID for the 'default' project on the Tableau server.
    'server'        specified server address
    'auth_token'    authentication token that grants user access to API calls
    'site_id'       ID of the site that the user is signed into
    """
    page_num, page_size = 1, 100   # Default paginating values

    # Builds the request
    url = server + "/api/{0}/sites/{1}/projects".format(VERSION, site_id)
    paged_url = url + "?pageSize={0}&pageNumber={1}".format(page_size, page_num)
    server_response = requests.get(paged_url, headers={'x-tableau-auth': auth_token})
    _check_status(server_response, 200)
    xml_response = ET.fromstring(_encode_for_display(server_response.text))

    # Used to determine if more requests are required to find all projects on server
    total_projects = int(xml_response.find('t:pagination', namespaces=xmlns).get('totalAvailable'))
    max_page = int(math.ceil(total_projects / page_size))

    projects = xml_response.findall('.//t:project', namespaces=xmlns)

    # Continue querying if more projects exist on the server
    for page in range(2, max_page + 1):
        paged_url = url + "?pageSize={0}&pageNumber={1}".format(page_size, page)
        server_response = requests.get(paged_url, headers={'x-tableau-auth': auth_token})
        _check_status(server_response, 200)
        xml_response = ET.fromstring(_encode_for_display(server_response.text))
        projects.extend(xml_response.findall('.//t:project', namespaces=xmlns))

    # Look through all projects to find the 'default' one
    for project in projects:
        if project.get('name') == 'default' or project.get('name') == 'Default':
            return project.get('id')
    raise LookupError("Project named 'default' was not found on server")

check this: https://github.com/tableau/rest-api-samples


In [6]:
server='http://203.205.27.73/'
username='biteam'
password='L,p7NE8a4B_)c`/RCk).'
site=""
VERSION = '3.4'
url = server + "/api/{0}/auth/signin".format(VERSION)

# Builds the request
xml_request = ET.Element('tsRequest')
credentials_element = ET.SubElement(xml_request, 'credentials', name=username, password=password)
ET.SubElement(credentials_element, 'site', contentUrl=site)
xml_request = ET.tostring(xml_request)

# Make the request to server
server_response = requests.post(url, data=xml_request)
_check_status(server_response, 200)

# ASCII encode server response to enable displaying to console
server_response = _encode_for_display(server_response.text)

# Reads and parses the response
parsed_response = ET.fromstring(server_response)

# Gets the auth token and site ID
token = parsed_response.find('t:credentials', namespaces=xmlns).get('token')
site_id = parsed_response.find('.//t:site', namespaces=xmlns).get('id')
print(token, site_id)

1vnmcEwXToa2JMwDnM3_9w|4zt3Q6HZgXxlRtGZeqxiNX5VAoPpQh8P a4821863-b74a-4ab9-91be-dcbaf98b9583


In [17]:
print(parsed_response)

<Element '{http://tableau.com/api}tsResponse' at 0x00000149A3373CC0>


In [81]:
workbook_file_path = os.path.abspath('C:/Users/DELL/Downloads/test1.twbx')
workbook_file = os.path.basename(workbook_file_path)
workbook_filename, file_extension = workbook_file.split('.', 1)

In [70]:
print("\n*Publishing '{0}' to the default project as {1}*".format(workbook_file, username))


*Publishing 'test1.twbx' to the default project as admin*


In [72]:
workbook_size = os.path.getsize(workbook_file_path)
chunked = workbook_size >= FILESIZE_LIMIT
print(workbook_size)
print(chunked)

533371
False


In [73]:
project_id="a2effea6-5c3c-40a7-ac6e-2870b4030c0d"

In [74]:
xml_request = ET.Element('tsRequest')
workbook_element = ET.SubElement(xml_request, 'workbook', name="workbook_filename_onserver")
ET.SubElement(workbook_element, 'project', id=project_id)
xml_request = ET.tostring(xml_request)

In [75]:
print(xml_request)

b'<tsRequest><workbook name="workbook_filename"><project id="a2effea6-5c3c-40a7-ac6e-2870b4030c0d" /></workbook></tsRequest>'


In [76]:
print("\n3. Publishing '" + workbook_file + "' using the all-in-one method (workbook under 64MB)")
# Read the contents of the file to publish
with open(workbook_file_path, 'rb') as f:
    workbook_bytes = f.read()

# Finish building request for all-in-one method
parts = {'request_payload': ('', xml_request, 'text/xml'),
         'tableau_workbook': (workbook_file, workbook_bytes, 'application/octet-stream')}
payload, content_type = _make_multipart(parts)

publish_url = server + "/api/{0}/sites/{1}/workbooks".format(VERSION, site_id)
publish_url += "?workbookType={0}&overwrite=true".format(file_extension)


3. Publishing 'test1.twbx' using the all-in-one method (workbook under 64MB)


In [78]:
print("\tUploading...")
server_response = requests.post(publish_url, data=payload, headers={'x-tableau-auth': token, 'content-type': content_type})
_check_status(server_response, 201)

	Uploading...


In [95]:
# data source uploadd

In [96]:
ds_file_path = os.path.abspath('C:/Users/DELL/Downloads/test.hyper')
ds_file = os.path.basename(workbook_file_path)
ds_filename, file_extension = workbook_file.split('.', 1)

In [97]:
ds_size = os.path.getsize(workbook_file_path)
chunked = ds_size >= FILESIZE_LIMIT
print(ds_size)
print(chunked)

65536
False


In [98]:
project_id="a2effea6-5c3c-40a7-ac6e-2870b4030c0d"

In [99]:
xml_request = ET.Element('tsRequest')
workbook_element = ET.SubElement(xml_request, 'datasource', name="ds_filename_onserver")
ET.SubElement(workbook_element, 'project', id=project_id)
xml_request = ET.tostring(xml_request)
xml_request

b'<tsRequest><datasource name="ds_filename_onserver"><project id="a2effea6-5c3c-40a7-ac6e-2870b4030c0d" /></datasource></tsRequest>'

In [103]:
print("\n3. Publishing '" + ds_file + "' using the all-in-one method (ds under 64MB)")
# Read the contents of the file to publish
with open(ds_file_path, 'rb') as f:
    ds_bytes = f.read()

# Finish building request for all-in-one method
parts = {'request_payload': ('', xml_request, 'text/xml'),
         'tableau_datasource': (ds_file, ds_bytes, 'application/octet-stream')}
payload, content_type = _make_multipart(parts)

publish_url = server + "/api/{0}/sites/{1}/datasources".format(VERSION, site_id)
publish_url += "?datasourceType={0}&overwrite=true".format(file_extension)
print(publish_url)


3. Publishing 'test.hyper' using the all-in-one method (ds under 64MB)
http://203.205.27.73//api/3.4/sites/a4821863-b74a-4ab9-91be-dcbaf98b9583/datasources?datasourceType=hyper&overwrite=true


In [104]:
print("\tUploading...")
server_response = requests.post(publish_url, data=payload, headers={'x-tableau-auth': token, 'content-type': content_type})
_check_status(server_response, 201)
print("\tDone")

	Uploading...


In [1]:
# Run flow

In [2]:
flow_id="64d49917-6560-4328-b765-5dfd34f48997"

In [7]:
xml_request = ET.Element('tsRequest')
xml_request = ET.tostring(xml_request)
xml_request

b'<tsRequest />'

In [9]:
publish_url = server + "/api/{0}/sites/{1}/flows/{2}/run".format(VERSION, site_id, flow_id)
print(publish_url)

http://203.205.27.73//api/3.4/sites/a4821863-b74a-4ab9-91be-dcbaf98b9583/flows/64d49917-6560-4328-b765-5dfd34f48997/run


In [14]:
server_response = requests.post(publish_url, data=xml_request, headers={'x-tableau-auth': token})
_check_status(server_response, 200)
# ASCII encode server response to enable displaying to console
server_response = _encode_for_display(server_response.text)
# Reads and parses the response
parsed_response = ET.fromstring(server_response)

In [29]:
jobid = parsed_response.find('t:job', namespaces=xmlns).get('id')
print(jobid+'---')
print(ET.tostring(parsed_response))

0ce5bde4-8068-48ac-b501-418bc0a36a5c---
b'<ns0:tsResponse xmlns:ns0="http://tableau.com/api" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://tableau.com/api http://tableau.com/api/ts-api-3.4.xsd"><ns0:job id="0ce5bde4-8068-48ac-b501-418bc0a36a5c" mode="Asynchronous" type="RunFlow" createdAt="2021-08-13T08:39:30Z"><ns0:runFlowJobType><ns0:flow id="64d49917-6560-4328-b765-5dfd34f48997" name="F_raw_data_test" /></ns0:runFlowJobType></ns0:job></ns0:tsResponse>'
